In [1]:
import math
import numpy as np
import scipy

In [16]:
def NIG(alpha, beta, gamma, nu, N=1000000):
    sigma2 = 1/np.random.gamma(shape=alpha, scale=1.0/beta, size=N)
    mu = np.random.normal(loc=gamma, scale=np.sqrt(sigma2 / nu))
    return mu, sigma2

def simulation(y, p, alpha, beta, gamma, nu):
    mu, _ = NIG(alpha, beta, gamma, nu)
    return np.power(np.abs(y - mu), p).mean()

def sim_kf(y, p, alpha, beta, gamma, nu):
    _, sigma2 = NIG(alpha, beta, gamma, nu)
    aa = (2 * sigma2 / nu) ** (p/2)
    bb = math.gamma((p+1) / 2) / math.pi ** 0.5
    cc = scipy.special.hyp1f1(-p/2, 1/2, -(gamma-y)**2 * nu / 2 / sigma2)
    return (aa * bb * cc).mean()

def ground_truth(y, p, alpha, beta, gamma, nu, n_max):
    aa = math.gamma((p+1) / 2) / math.pi ** 0.5 / math.gamma(alpha)
    bb = (2 * beta / nu) ** (p / 2)
    cc = 0
    for n in range(0, n_max+1):
        cc1 = np.prod([(m - p / 2) for m in range(n)])
        cc2 = math.gamma(alpha + n - p/2) / math.factorial(2 * n)
        cc3 = (-2 * (y - gamma)**2 * nu / beta) ** n
        cc += (cc1 * cc2 * cc3)
    return aa * bb * cc

def ground_truth_p2(y, alpha, beta, gamma, nu):
    return (y - gamma) ** 2 + beta / nu / (alpha-1)

In [26]:
mu, sigma2 = NIG(alpha, beta, gamma, nu, N=1000000)

In [52]:
x = (gamma-y)**2 * nu / 2 / sigma2

In [59]:
scipy.special.hyp1f1(-p/2, 1/2, -x)

array([ 2.87026351,  5.39155339,  9.32384027, ..., 10.33198575,
        2.8095857 ,  9.22561042])

In [58]:
np.exp(-x) * scipy.special.hyp1f1(-p/2, 1/2, x)

array([-0.27840064, -0.06713977, -0.01915027, ..., -0.0154178 ,
       -0.28164082, -0.01958655])

In [63]:
sigma2

array([1.07976083, 0.30395921, 0.10163721, ..., 0.08277035, 1.12826876,
       0.10381311])

In [61]:
ans = 0
for n in range(10):
    aa = np.prod([(m - p / 2) for m in range(n)])
    bb = math.gamma(1/2) / math.gamma(1/2 + n)
    cc = ((gamma - y) ** 2 * nu / 2 / sigma2) ** n
    dd = 1 / math.factorial(n)
    this = aa * bb * cc * dd
    ans += this
    print(f'n={n} this={this.mean():.2f} cum={ans.mean():.2f}')

n=0 this=1.00 cum=1.00
n=1 this=-15.75 cum=-14.75
n=2 this=-56.10 cum=-70.85
n=3 this=-302.79 cum=-373.64
n=4 this=-1762.01 cum=-2135.65
n=5 this=-10453.25 cum=-12588.90
n=6 this=-62112.17 cum=-74701.07
n=7 this=-366490.96 cum=-441192.03
n=8 this=-2135139.46 cum=-2576331.49
n=9 this=-12228528.14 cum=-14804859.63


In [48]:
beta, nu = 0.5, 0.9
gamma = -0.5
y = 2
p = 1
alpha = p/2 + 1.3
assert alpha > p / 2

In [23]:
sim_kf(y, p, alpha, beta, gamma, nu)

17.945067601614305

In [24]:
gt = ground_truth(y, p, alpha, beta, gamma, nu, n_max=10)
gt

193910.5068940935

In [25]:
sim = simulation(y, p, alpha, beta, gamma, nu)
sim

17.93260278198373